# emTools

### a Python module to interact with SerialEM to enable automated Transmission Electron Microscopy

## Installation instructions

### Prepare your PC for the use of Python and emTools

- first we will install Python and then use it to fetch the packages needed for emTools to work.
- Install Anaconda from [here](https://www.anaconda.com/download/ "Download Anaconda"). EmTools has been tested to work with both Python 2 and 3. Choose the appropriate OS.
- Start the Anaconda [Navigator](http://docs.anaconda.com/anaconda/user-guide/getting-started/ "Getting started with Anaconda").
- as explained in [section 4](http://docs.anaconda.com/anaconda/user-guide/getting-started/#run-python-in-a-jupyter-notebook "Run Python in a Jupyter Notebook"), install and start a Jupyter notebook.
- Download this [tutorial file](https://git.embl.de/schorb/serialem_tools/raw/master/emTools.ipynb) (use "save link as..."), open it in your Jupyter session and follow the instructions.


----
### How to use this jupyter notebook

In this document there are text blocks and code blocks that can be edited and executed.
In order to run a code block, click into it to activate and then press `CTRL+ Enter`. There is a Python logo and text on the top right of the notebook that tells you whether there are still operations running.

### Install necessary dependencies in Python

The following code block uses Anaconda to fetch necessary packages. Start the code by pressing `CTRL+ Enter` inside the code block.

In [ ]:
# this will load the package needed for python package management
import conda.cli

# this will install the dependencies
conda.cli.main('conda', 'install',  '-y', 'numpy', 'scipy', 'pandas', 'scikit-image', 'git', 'pip')
conda.cli.main('conda', 'install',  '-y', '-c conda-forge', 'tifffile')

## install/update emTools

Now we can install emTools using pip, another package manager for Python. Start by pressing `CTRL+ Enter` inside the code block. 

To update to the current version, simply execut this code block anytime.

In [ ]:
# this will load the pip package needed for python package management
from pip._internal import main as pipmain

# this will install the current version of emTools
pipmain(['install','git+https://git.embl.de/schorb/serialem_tools'])